<a href="https://colab.research.google.com/github/mauricionoris/25B3_ml/blob/master/colabs/xg_boost_with_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost —>  O que é

**XGBoost (eXtreme Gradient Boosting)** é uma implementação altamente otimizada do **gradient boosting** sobre **árvores de decisão**. Ele constrói o modelo de forma **aditiva**: começa simples e, a cada iteração (“árvore fraca”), corrige os **erros residuais** das iterações anteriores. O diferencial do XGBoost é combinar:

* **Regularização explícita** (controle de complexidade da árvore),
* **Segundo-ordem do gradiente** (usa gradiente e **hessiano**),
* **Amostragens** de linhas e colunas,
* **Treinamento paralelo**, suporte a **GPU** e dados **esparsos**,
* **Parada antecipada** e recursos extras (monotonicidade, tratamento de categorias, etc.).

---

## Função objetivo (visão matemática)

O modelo final é uma soma de árvores:

$$
\hat{y}_i = \sum_{t=1}^{T} f_t(x_i), \quad f_t \in \mathcal{F} \;(\text{árvores})
$$

A cada iteração $t$, minimiza-se:

$$
\mathcal{L}^{(t)} \approx \sum_{i=1}^{n} \left[ g_i w(x_i) + \tfrac{1}{2} h_i w(x_i)^2 \right] + \Omega(f_t)
$$

onde $g_i$ e $h_i$ são **gradiente** e **hessiano** da perda em relação à predição anterior; $w(x_i)$ é o incremento previsto pela nova árvore; e

$$
\Omega(f) = \gamma \cdot T_{\text{nós}} + \tfrac{\lambda}{2}\sum_{j} w_j^2
$$

penaliza número de folhas ($\gamma$) e pesos das folhas ($\lambda$).

Para cada **folha** com soma de gradientes $G$ e hessianos $H$, o peso ótimo é:

$$
w^* = -\frac{G}{H + \lambda}
$$

e o **ganho** de um **split** (quebra) é:

$$
\text{Gain} = \tfrac{1}{2} \left(\frac{G_L^2}{H_L+\lambda} + \frac{G_R^2}{H_R+\lambda} - \frac{G^2}{H+\lambda}\right) - \gamma
$$

(Se o ganho $\le 0$, não vale a pena dividir.)

---

## Como ele constrói as árvores

1. **Calcula** $g_i$ e $h_i$ (1ª e 2ª derivadas da perda) em todos os exemplos.
2. **Procura quebras** que maximizem o **ganho** (acima). Pode usar método **hist** (mais rápido e padrão atual) ou **gpu\_hist**.
3. **Aplica regularização** para evitar overfitting (parâmetros $\gamma, \lambda, \alpha$).
4. **Adiciona** a árvore ao conjunto com um **learning rate** ($\eta$, “shrinkage”).
5. **Repete** até atingir $n\_estimators$ ou **early stopping**.

---

## Por que ele é rápido/robusto

* **Histograma/quantis** para procurar splits (menor custo).
* **Suporte a esparsidade** (faltantes e zeros tratados com caminhos padrão na árvore).
* **Subamostragem** de linhas ( `subsample` ) e colunas ( `colsample_bytree/by_level/by_node` ).
* **Paralelização** e **GPU** (`tree_method='gpu_hist'`).
* **Regularização** forte na estrutura da árvore.

---

## Hiperparâmetros (o que mais importa)

* **`n_estimators`**: número de árvores. Quanto maior, melhor, **até** overfitting; combine com `learning_rate`.
* **`learning_rate` (η)**: passo de cada árvore. Valores **menores (0.01–0.1)** costumam generalizar melhor, mas exigem mais árvores.
* **`max_depth` / `max_leaves`**: controla complexidade. Árvores mais rasas/leaves limitadas ajudam a generalizar.
* **`min_child_weight`**: mínimo de hessiano por folha; aumenta → modelo mais conservador.
* **`gamma`**: ganho mínimo para dividir; maior → menos splits.
* **`subsample`, `colsample_bytree`**: 0.6–0.9 são bons pontos de partida.
* **`reg_lambda`, `reg_alpha`**: L2 e L1 nos pesos; reduzem overfitting.
* **`tree_method`**: `hist` (padrão rápido), `gpu_hist` (se GPU).
* **Classificação desbalanceada**: **`scale_pos_weight = n_neg/n_pos`** ajuda bastante.

> Dica de prática: fixe `n_estimators` alto (p.ex. 2000) e use **`early_stopping_rounds`** com validação para “parar” no ponto ótimo.

---

## Quando usar (e quando evitar)

* **Usar**: tabulares pequenos a médios, com mistura de numéricas/categóricas, dados esparsos, relações não lineares, competição de Kaggle.
* **Evitar/avaliar**: dados massivos com milhões de linhas + muitas categorias de alta cardinalidade (às vezes **CatBoost** lida melhor com categorias), ou quando você pode explorar representações em sequência/imagem (redes neurais).

### Catboost: [Referência](https://catboost.ai/docs/en/concepts/educational-materials-papers)

---

## Boas práticas rápidas

* Separe **validação** e use **early stopping**.
* Ajuste primeiro **taxa de aprendizado** e **n\_estimators**, depois **profundidade** e **regularização**.
* Para **classes desbalanceadas**, ajuste `scale_pos_weight` E avalie por **AUC/PR**, **F1**, não só acurácia.
* Avalie **importâncias** com **permutation** ou **SHAP** (as importâncias “gain”/“weight” podem enganar).
* Trate categorias: versões recentes suportam **categorical nativo** (ou use one-hot/target encoding); valide o que funciona melhor no seu dado.

---

## Exemplos de código

### (A) API estilo scikit-learn — classificação binária

```python
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    min_child_weight=1.0,
    objective="binary:logistic",
    tree_method="hist",           # use "gpu_hist" se tiver GPU
    eval_metric="auc",
    n_jobs=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=False,
    early_stopping_rounds=100
)

pred_proba = model.predict_proba(X_valid)[:, 1]
print("AUC:", roc_auc_score(y_valid, pred_proba))
```

**Desbalanceamento:**

```python
# Regra prática: n_neg / n_pos
from collections import Counter
cnt = Counter(y_train)
scale = cnt[0] / cnt[1]
model.set_params(scale_pos_weight=scale)
```

### (B) Regressão com DMatrix (API “baixa”)

```python
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

params = {
    "objective": "reg:squarederror",
    "eta": 0.05,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "lambda": 1.0,
    "alpha": 0.0,
    "tree_method": "hist",
    "eval_metric": "rmse"
}

watchlist = [(dtrain, "train"), (dvalid, "valid")]
bst = xgb.train(params, dtrain, num_boost_round=5000, evals=watchlist, early_stopping_rounds=200)
```

### (C) Categorias nativas (pandas `category`)

```python
import pandas as pd
X_cat = X.copy()
for col in cat_cols:
    X_cat[col] = X_cat[col].astype("category")

model = XGBClassifier(
    tree_method="hist",
    enable_categorical=True,   # suporta divisão por categoria sem one-hot
    # demais parâmetros...
)
model.fit(X_cat, y)
```

### (D) Restrições monotônicas (útil em risco/crédito)

```python
# Ex.: y deve crescer com feature1 e decrescer com feature2
# ordem das restrições segue a ordem das colunas em X
model = XGBRegressor(
    monotone_constraints="(1,-1,0,0)",  # 1: crescente, -1: decrescente, 0: livre
    tree_method="hist",
    # demais parâmetros...
)
```

### (E) Importância & SHAP (interpretação)

```python
# Importância padrão
import matplotlib.pyplot as plt
xgb.plot_importance(model, max_num_features=15)
plt.show()

# SHAP (melhor para explicabilidade global/local)
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_valid)
shap.summary_plot(shap_values, X_valid)  # requer ambiente gráfico
```

---

## Hiperparâmetros — valores de partida (heurística)

* Classificação:

  * `learning_rate=0.05`, `n_estimators=2000`, `early_stopping_rounds=100–300`
  * `max_depth=4–8` (ou usar `max_leaves` \~ 31–255)
  * `subsample=0.8`, `colsample_bytree=0.8`
  * `min_child_weight=1–5`, `gamma=0–5`
  * `reg_lambda=1–5`, `reg_alpha=0–1`
* Regressão:

  * Parecido; ajuste `eval_metric` para `rmse`/`mae`.

---

## Erros comuns (e como evitar)

* **Acurácia em dado desbalanceado** → use **AUC-PR**, **F1**, **Recall**; ajuste `scale_pos_weight`.
* **Overfitting com `max_depth` alto** → aumente regularização e use early stopping.
* **Parar cedo demais** → `early_stopping_rounds` muito baixo; aumente e valide.
* **Importâncias ingênuas** → prefira **Permutation** / **SHAP**.
* **Leakage** (ex.: variáveis pós-evento) → reavalie features e validação temporal.

---

## Resumo

* XGBoost = **gradient boosting** + **regularização forte** + **segundo-ordem** + **engenharia de velocidade**.
* Brilha em **dados tabulares** e entrega **ótima performance** com bons defaults.
* Foque em: **learning rate + early stopping**, **profundidade/folhas**, **regularização**, **subamostragens**, e métricas corretas.



# Pseudocode

```python
Entrada:
    - Dados de treino: X = {x₁,...,xₙ}, rótulos y = {y₁,...,yₙ}
    - Função de perda L(y, ŷ)
    - Número de árvores T
    - Taxa de aprendizado η
    - Parâmetros de regularização (λ, γ, etc.)

Inicialização:
    - Definir predição inicial ŷᵢ⁽⁰⁾ (ex: média de y no caso de regressão log odds no caso de classificação)

Para t = 1 até T:   # loop de boosting
    1. Calcular gradiente e hessiano para cada exemplo i:
           gᵢ = ∂L(yᵢ, ŷᵢ⁽ᵗ⁻¹⁾) / ∂ŷᵢ
           hᵢ = ∂²L(yᵢ, ŷᵢ⁽ᵗ⁻¹⁾) / ∂ŷᵢ²

    2. Construir uma árvore de decisão fₜ(x):
        - Começar com um nó raiz contendo todos os exemplos
        - Enquanto possível:
            Para cada variável de entrada e ponto de corte candidato:
                - Calcular somas G, H de gradientes e hessianos nos grupos esquerdo (L) e direito (R) (VEJA DETALHES ABAIXO)
                - Calcular ganho:
                      Gain = ½ [ G_L² / (H_L + λ) + G_R² / (H_R + λ) - G² / (H + λ) ] - γ
                - Escolher split com maior ganho positivo
            Se ganho ≤ 0: não dividir mais o nó

        - Para cada folha j da árvore:
            - Calcular peso ótimo:
                  wⱼ* = - Gⱼ / (Hⱼ + λ)

    3. Atualizar predições:
           ŷᵢ⁽ᵗ⁾ = ŷᵢ⁽ᵗ⁻¹⁾ + η ⋅ fₜ(xᵢ)

Saída:
    - Modelo final: ŷ(x) = Σₜ η ⋅ fₜ(x)


```


Referência: [O que é uma matrix hessiana](https://www.datacamp.com/pt/tutorial/hessian-matrix)

#Contexto

Quando estamos em um nó de uma árvore em construção, temos um **conjunto de exemplos** $S$ que caíram nesse nó.
Para decidir se devemos **dividir** o nó em dois grupos (esquerda L e direita R), o XGBoost olha para o quanto essa divisão **reduz a perda** esperada.

A perda aproximada é baseada em **gradientes** e **hessianos** de cada exemplo em relação à predição atual.

---

# Gradiente e Hessiano

Para cada exemplo $i$ no nó:

* **Gradiente**

$$
g_i = \frac{\partial L(y_i, \hat{y}_i)}{\partial \hat{y}_i}
$$

Indica a direção do erro (positivo/negativo).

* **Hessiano**

$$
h_i = \frac{\partial^2 L(y_i, \hat{y}_i)}{\partial \hat{y}_i^2}
$$

Indica a “curvatura” da perda (quanto confiar no gradiente).

---

# Somas G e H

Agora, dentro de um nó:

* Para todos os exemplos $i \in S$:

$$
G = \sum_{i \in S} g_i, \quad H = \sum_{i \in S} h_i
$$

* Se fazemos uma divisão (split), separamos os exemplos em **lado esquerdo (L)** e **lado direito (R)**:

$$
G_L = \sum_{i \in L} g_i, \quad H_L = \sum_{i \in L} h_i
$$

$$
G_R = \sum_{i \in R} g_i, \quad H_R = \sum_{i \in R} h_i
$$

E claro:

$$
G = G_L + G_R, \quad H = H_L + H_R
$$

---

# Como isso é usado

O XGBoost calcula o **ganho da divisão** assim:

$$
\text{Gain} = \tfrac{1}{2} \left( \frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{G^2}{H + \lambda} \right) - \gamma
$$

* **Primeiro termo:** qualidade de deixar só os exemplos à esquerda.
* **Segundo termo:** qualidade de deixar só os exemplos à direita.
* **Terceiro termo:** qualidade do nó antes de dividir (baseline).
* Subtraímos $\gamma$ = penalidade por criar um novo split.

Se o **ganho > 0**, a divisão ajuda → o split é aceito.
Se **ganho ≤ 0**, o XGBoost não divide.

---

# Intuição

* **$G$** mede “quanto erro acumulado” tem naquele grupo (via gradientes).
* **$H$** mede “quanta informação de curvatura” temos (ajuda a estabilizar).
* Dividir em L e R significa redistribuir o erro em duas folhas, cada uma com seu peso ótimo.
* O ganho mede se essa redistribuição **reduz a perda global** mais do que custa criar um novo nó.




XGBoost, Decision Tree e Random Forest são todos **métodos baseados em árvores de decisão**, mas com filosofias bem diferentes.

Abaixo a comparação em **estrutura**, **como aprendem**, **vantagens** e **desvantagens**.


# **Decision Tree (árvore única)**

* **Como funciona:**

  * Divide o espaço de atributos em regiões, escolhendo splits que maximizam algum critério (ex.: Gini, entropia, MSE).
  * Cada folha dá uma predição (classe majoritária ou média da variável alvo).
* **Pontos fortes:**

  * Simples de entender e visualizar.
  * Não precisa de muita preparação dos dados (normalização, linearidade, etc.).
  * Interpretação intuitiva.
* **Pontos fracos:**

  * Muito instável: pequenas mudanças nos dados podem gerar árvores muito diferentes.
  * Alta tendência a **overfitting** (se não for podada ou limitada em profundidade).
  * Pouco competitiva sozinha em datasets complexos.

---

# **Random Forest (ensemble bagging de árvores)**

* **Como funciona:**

  * Constrói **muitas árvores de decisão independentes** em subconjuntos **bootstrap** dos dados (bagging).
  * Em cada split da árvore, considera **apenas um subconjunto aleatório de variáveis**.
  * Predição final: média (regressão) ou votação (classificação).
* **Pontos fortes:**

  * Reduz **variância** em relação a uma única árvore → mais estável.
  * Lida bem com dados de alta dimensionalidade.
  * Funciona bem em muitos problemas “out of the box”.
* **Pontos fracos:**

  * Cada árvore é profunda → menos interpretável.
  * Modelo pesado (centenas/milhares de árvores).
  * Correlações entre árvores ainda limitam a melhora de performance.
  * Não tem mecanismo explícito de correção de viés.

---

# **XGBoost (gradient boosting de árvores)**

* **Como funciona:**

  * Constrói árvores **sequencialmente** (não em paralelo).
  * Cada nova árvore tenta corrigir os **erros residuais** das anteriores, usando **gradiente + hessiano** da função de perda.
  * Usa **regularização forte** ($\lambda, \gamma, \alpha$) para evitar overfitting.
  * Implementa otimizações (histogramas, sparsity-aware, GPU, etc.).
* **Pontos fortes:**

  * Altíssima performance em dados tabulares.
  * Melhor controle de overfitting do que RF.
  * Suporta customização de função de perda (classificação, regressão, ranking).
  * Tem recursos avançados (tratamento de categorias, restrições monotônicas, early stopping).
* **Pontos fracos:**

  * Mais difícil de **tunar** (hiperparâmetros sensíveis).
  * Mais lento de treinar que uma árvore ou até um RF simples (mas ainda muito otimizado).
  * Menos interpretável diretamente (precisa de SHAP, feature importance, etc.).

---

# Comparação direta

| Característica       | Decision Tree       | Random Forest               | XGBoost (Boosting)                     |
| -------------------- | ------------------- | --------------------------- | -------------------------------------- |
| Estrutura            | Uma árvore          | Muitas árvores, em paralelo | Muitas árvores, em sequência           |
| Combate overfitting  | Podas/profundidade  | Bagging + voto              | Regularização + boosting               |
| Variância            | Alta                | Baixa                       | Média-baixa (regularizada)             |
| Viés                 | Baixo (se profunda) | Médio (média de árvores)    | Muito baixo (corrige iterativamente)   |
| Velocidade de treino | Muito rápido        | Médio (paralelizável)       | Mais lento (sequencial, mas otimizado) |
| Interpretação        | Alta                | Média-baixa                 | Baixa (precisa SHAP, etc.)             |
| Performance típica   | Baixa-média         | Boa                         | Muito alta (SOTA em tabulares)         |

---

# Intuição comparativa

* **Decision Tree** = modelo simples, fácil de entender, mas sofre com overfitting.
* **Random Forest** = “comitê de árvores” → reduz variação, mais robusto, funciona bem quase sempre.
* **XGBoost** = “time de árvores em série” → cada nova árvore corrige erros das anteriores → atinge performance muito superior, mas exige mais cuidado.





# Exemplo


Este é um notebook que investiga a amostra xgboost do Titanic.

In [23]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.base import TransformerMixin

In [3]:
train_df = pd.read_csv('train.csv', header=0)
test_df = pd.read_csv('test.csv', header=0)

In [12]:
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

No scikit-learn, cada método de aprendizado tem o mesmo nome.
* fit(): **ajusta dados de treinamento**. Para aplicações de aprendizado supervisionado, aceita dois argumentos: o dado X e os rótulos y (por exemplo, model.fit(X, y)).
* transform(): Para algoritmos de seleção de recursos, reduz o conjunto de dados aos recursos selecionados. Para alguns modelos de classificação e regressão, como modelos lineares e florestas aleatórias, **este método reduz o conjunto de dados aos recursos mais informativos.** Esses modelos de classificação e regressão também podem ser usados como métodos de seleção de recursos.

Detalhes em: https://github.com/amueller/scipy_2015_sklearn_tutorial/blob/master/notebooks/02.5%20Review%20of%20Scikit-learn%20API.ipynb

In [7]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
feature_columns_to_use = ['Pclass','Sex','Age','Fare','Parch'] # parch - Num Pais e Crianças a bordo
nonnumeric_columns = ['Sex']

big_X = pd.concat((train_df[feature_columns_to_use],test_df[feature_columns_to_use]))
big_X_imputed = DataFrameImputer().fit_transform(big_X)

big_X_imputed.head(7)


,Pclass,Sex,Age,Fare,Parch
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,0
2,3,female,26.0,7.9250,0
3,1,female,35.0,53.1000,0
4,3,male,35.0,8.0500,0
5,3,male,28.0,8.4583,0
6,1,male,54.0,51.8625,0


fit_transform(): Depois de executar fit(), execute transform() nos mesmos dados para preencher NaNs com o modo ou a mediana.

In [15]:
le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

big_X_imputed.head(3)

,Pclass,Sex,Age,Fare,Parch
0,3,1,22.0,7.2500,0
1,1,0,38.0,71.2833,0
2,3,0,26.0,7.9250,0


Codificador de etiqueta

In [18]:
train_X = big_X_imputed[0:train_df.shape[0]].values
test_X = big_X_imputed[train_df.shape[0]::].values
train_y = train_df['Survived']

Os dados adicionados para imputação de valores ausentes foram separados novamente em treinamento e teste. Além disso, a coluna "Sobreviveu" foi rotulada como "train_y".

In [19]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)


In [20]:
predictions = gbm.predict(test_X)

Ao usar o xgboost, tudo o que você precisa fazer é criar uma classe e executar fit->predict.

In [22]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'], 'Survived': predictions })
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


# Exercício — Sobrevivência no Titanic com Árvores, Random Forest e XGBoost

## Objetivo

Prever quais passageiros sobreviveram ao naufrágio do Titanic e **comparar** três abordagens:

1. **Decision Tree**, 2) **Random Forest**, 3) **XGBoost**.
   Você deve **treinar, validar, comparar, explicar** e **reportar** os resultados.

## Dados

* Use o dataset clássico **Titanic** (train/test) — as colunas típicas incluem: `Survived` (alvo), `Pclass`, `Sex`, `Age`, `SibSp`, `Parch`, `Fare`, `Embarked`, `Cabin`, `Ticket`, `Name`, `PassengerId`.

## Tarefas

### 1) Preparação

* Carregue `train.csv`.
* Separe `X` e `y` (`y = Survived`).
* Crie **conjuntos**: treino/validação com `train_test_split(test_size=0.2, stratify=y, random_state=42)`.

### 2) Pré-processamento (via `Pipeline`)

* Numéricas: `Age`, `Fare`, `SibSp`, `Parch` → imputar (mediana) e **opcionalmente** padronizar (não é obrigatório para árvores).
* Categóricas: `Sex`, `Embarked`, `Pclass` (tratar como categoria), e **opcionalmente** derivar:

  * `Title` a partir de `Name` (Sr., Sra., etc.),
  * `FamilySize = SibSp + Parch + 1`,
  * `IsAlone = 1{FamilySize == 1}`,
  * extrair deck de `Cabin` (primeira letra).
* Use `OneHotEncoder(handle_unknown="ignore")` para categóricas.
* Deixe tudo em um `ColumnTransformer`.

### 3) Modelagem

Implemente **3 pipelines** (mesmo pré-processamento + estimador):

**A. Decision Tree**

* Hiperparâmetros a buscar: `max_depth`, `min_samples_split`, `min_samples_leaf`, `ccp_alpha`.

**B. Random Forest**

* Buscar: `n_estimators`, `max_depth` (ou `max_leaf_nodes`), `min_samples_leaf`, `max_features` (ex.: `sqrt`, `log2`, float), `bootstrap`.

**C. XGBoost**

* Estimador: `xgboost.XGBClassifier(tree_method="hist" ou "gpu_hist" se disponível, eval_metric="auc")`.
* Buscar: `n_estimators`, `learning_rate`, `max_depth` **ou** `max_leaves`, `subsample`, `colsample_bytree`, `reg_lambda`, `reg_alpha`, `min_child_weight`.

Use `GridSearchCV` (ou `RandomizedSearchCV`) com **validação estratificada** e **métrica principal** = `roc_auc`. Salve o melhor conjunto de hiperparâmetros de cada modelo.

### 4) Avaliação (no conjunto de validação)

Para **cada modelo**:

* Métricas: **ROC AUC**, **Accuracy**, **Precision**, **Recall**, **F1**.
* Curva ROC e AUC; **curva Precisão-Revocação** (se quiser aprofundar).
* **Matriz de confusão** com cutoff padrão 0.5 (e discuta se um limiar diferente ajudaria).
* **Overfitting check**: compare desempenho médio em CV versus validação hold-out.
* **Tempo de treino** (aproximado) e contagem de features após one-hot.

### 5) Interpretabilidade e diagnóstico

* **Importância de features**:

  * Tree/RF: `feature_importances_` e **Permutation Importance** (recomendado).
  * XGBoost: `feature_importances_` (gain) e, se possível, **SHAP** para `summary_plot` (global) e 2–3 explicações locais.
* **Análise de erros**:

  * Liste 5–10 exemplos **falsos negativos** e **falsos positivos** mais “confiantes” (probabilidade alta, mas erro).
  * Discuta possíveis razões (ex.: idade faltante, efeito de `Sex`, `Pclass`, `Fare` alto etc.).
* **Calibração** (opcional): **Reliability curve** ou `CalibratedClassifierCV`.

### 6) Comparação final

* Tabela comparativa com as métricas dos três modelos.
* Gráfico (barras) comparando **ROC AUC** dos três.
* Conclusão curta: quem venceu? por quê? quais features mais impactaram? houve overfitting?

---

## Esqueleto de código (mínimo) — ajuste à vontade

```python
# 1) setup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

df = pd.read_csv("train.csv")
y = df["Survived"]
X = df.drop(columns=["Survived"])

# 2) feature engineering simples (exemplos; adicione os seus)
X["FamilySize"] = X["SibSp"] + X["Parch"] + 1
X["IsAlone"] = (X["FamilySize"] == 1).astype(int)
X["Title"] = X["Name"].str.extract(r",\s*([^\.]+)\.", expand=False)

num_cols = ["Age","Fare","SibSp","Parch","FamilySize"]
cat_cols = ["Sex","Embarked","Pclass","Title"]  # trate Pclass como categoria

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

num_tf = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="median")),
    # árvores não exigem scaler; se quiser, adicione StandardScaler
])

cat_tf = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("oh", OneHotEncoder(handle_unknown="ignore"))
])

pre = ColumnTransformer([
    ("num", num_tf, num_cols),
    ("cat", cat_tf, cat_cols),
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def fit_and_eval(model, param_grid, name):
    pipe = Pipeline([("pre", pre), ("clf", model)])
    gs = GridSearchCV(
        pipe, param_grid=param_grid, cv=cv, scoring="roc_auc", n_jobs=-1, verbose=0
    )
    gs.fit(X_train, y_train)
    best = gs.best_estimator_
    proba = best.predict_proba(X_valid)[:,1]
    pred = (proba >= 0.5).astype(int)

    metrics = {
        "model": name,
        "roc_auc": roc_auc_score(y_valid, proba),
        "accuracy": accuracy_score(y_valid, pred),
        "precision": precision_score(y_valid, pred),
        "recall": recall_score(y_valid, pred),
        "f1": f1_score(y_valid, pred),
        "best_params": gs.best_params_
    }
    return best, metrics

# 3) grids (comece pequeno e amplie)
tree_grid = {
    "clf__max_depth": [3,5,7,9],
    "clf__min_samples_leaf": [1,2,5,10],
    "clf__ccp_alpha": [0.0, 0.001, 0.01]
}

rf_grid = {
    "clf__n_estimators": [200, 500],
    "clf__max_depth": [None, 6, 10],
    "clf__min_samples_leaf": [1,2,5],
    "clf__max_features": ["sqrt", "log2", 0.5],
    "clf__bootstrap": [True]
}

xgb_grid = {
    "clf__n_estimators": [400, 800, 1200],
    "clf__learning_rate": [0.05, 0.1],
    "clf__max_depth": [3,4,5],
    "clf__subsample": [0.8, 1.0],
    "clf__colsample_bytree": [0.8, 1.0],
    "clf__reg_lambda": [1.0, 3.0, 5.0],
    "clf__reg_alpha": [0.0, 0.5],
    "clf__min_child_weight": [1, 3]
}

best_tree, m_tree = fit_and_eval(DecisionTreeClassifier(random_state=42), tree_grid, "DecisionTree")
best_rf, m_rf = fit_and_eval(RandomForestClassifier(random_state=42), rf_grid, "RandomForest")
best_xgb, m_xgb = fit_and_eval(XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    random_state=42,
    n_jobs=-1
), xgb_grid, "XGBoost")

results = pd.DataFrame([m_tree, m_rf, m_xgb])
print(results[["model","roc_auc","accuracy","precision","recall","f1"]])
print("\nMelhores hiperparâmetros:")
for m in [m_tree, m_rf, m_xgb]:
    print(m["model"], "->", m["best_params"])

# 4) curvas e confusão (ex.: melhor modelo)
best_model = max([(best_tree,m_tree),(best_rf,m_rf),(best_xgb,m_xgb)], key=lambda t: t[1]["roc_auc"])[0]
proba = best_model.predict_proba(X_valid)[:,1]
pred = (proba>=0.5).astype(int)

RocCurveDisplay.from_predictions(y_valid, proba)
plt.show()
PrecisionRecallDisplay.from_predictions(y_valid, proba)
plt.show()
print("Matriz de confusão:\n", confusion_matrix(y_valid, pred))
```

> Dica: para **Permutation Importance** use `sklearn.inspection.permutation_importance`. Para **SHAP**, instale `shap` e use `TreeExplainer` no `best_xgb`.

---

## O que entregar

1. **Notebook** com:

   * Pré-processamento (e justificativas).
   * Busca de hiperparâmetros (grids escolhidos).
   * Tabela comparativa de métricas.
   * Curvas ROC/PR e matriz de confusão.
   * Importância de features (Permutation e/ou SHAP).
   * Análise de erros (FN/FP mais confiantes).
   * Conclusão: **qual modelo escolheu e por quê**.

